# Minimum working example for dual zonotopes
Whirlwind tour of the codebase with most of the functionality explained (for the naive case)

Table of contents:
1. Loading a net and setting up an example verification problem 
2. Computing preactivation bounds / {boxes, zonotopes, polytopes}
3. Playing with the naive dual object 
4. Playing with the decomposed dual object
5. Interacting with zonotopes/partitioning

In [1]:
# Basic import block 
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt


import abstract_domains as ad 
from neural_nets import FFNet, PreactBounds
import train
import utilities
import dual_naive as dn
import dual_decompose as dd 
import pickle
import seaborn as sns 
sns.set()

valsum = lambda d: sum(d.values()) # handy little function
flatten = lambda lol: [subel for sublist in lol for subel in sublist]

torch.manual_seed(42)

# Part 1: Loading a net and setting up an example

In [2]:
### Load and train a net
# Simple [784, 256, 128, 64, 10] PGD-trained MNIST network

make_net = lambda: FFNet(
                    nn.Sequential(nn.Linear(784, 256), 
                    nn.ReLU(), 
                    nn.Linear(256, 128), 
                    nn.ReLU(), 
                    nn.Linear(128, 64), 
                    nn.ReLU(), 
                    nn.Linear(64, 10)))


adv_net = make_net() # Make network

mnist_train = train.load_mnist_data('train', batch_size=128) # load datasets
mnist_val = train.load_mnist_data('val')


headless_atk = train.PGD(None, float('inf'), 0.1, 10, lb=0.0, ub=1.0) #setup attack params
advtrain_params = train.TrainParameters(mnist_train, mnist_val, 10, adv_attack=headless_atk) # setup train params


try: # Try to load the pickled network, otherwise train it
    adv_net = pickle.load(open('adv_net.pkl', 'rb'))
except:
    train.training_loop(adv_net, advtrain_params)
    pickle.dump(adv_net, open('adv_net.pkl', 'wb'))
    train.test_validation(adv_net, advtrain_params)

advtrain_params.adv_attack = train.PGD(adv_net, float('inf'), 0.1, 10, lb=0.0, ub=1.0)
print('Clean acc: %.2f; Robust acc: %.2f' % train.test_validation(adv_net, advtrain_params)[1:])

Clean acc: 98.48; Robust acc: 90.01


In [3]:
'''
Now convert network to a binary classifier: (can only do binary classifier certification).
We'll do the following jointly:
- pick an MNIST example to certify 
- build the Hyperbox that defines the adversarial input region (what the adv can do)
- build a Binary classifier of <label> vs <label + 1>  (e.g., if the example is a 7, this is a 7vs8 classifier)
'''



def setup_ex(x, network, rad): # handy function that does the steps above^
    # Returns bin_net, input_domain 
    test_box = ad.Hyperbox.linf_box(x.view(-1), rad) 
    ypred = network(x.view(1, -1)).max(dim=1)[1].item()
    
    bin_net = network.binarize(ypred, ((ypred +1) % 10))
    return bin_net, test_box


RAD = 0.1
test_ex = next(iter(mnist_val))[0][20].view(-1) #Just pick an arbitrary example
bin_net, test_input = setup_ex(test_ex, adv_net, RAD)
#test_input = test_input.clamp(0.0, 1.0)
print(bin_net(test_ex.view(1, 28, 28)))

tensor([[13.5886]], grad_fn=<AddmmBackward>)


# Part 2: Computing Preactivation Bounds
Explaining the Abstract Domain framework I've rebuilt for this


In [4]:
""" All this stuff is contained in abstract_domains.py for extensions of the base AbstractDomain class.
    
    Ultimately we want to compute PreactBounds object which has the intermediate bounds stored in a list.
    The API is simple, see below for boilerplate methods for computing preactivation bounds
"""

def get_hyperbox_prespec(net, test_input):
    # Hyperbox bounds (interval bounds)
    bounds = PreactBounds(net, test_input, ad.Hyperbox)
    bounds.compute() 
    return bounds 

def get_zonobox_prespec(net, test_input):
    # Computes zonotopes, but then converts to hyperboxes
    bounds = PreactBounds(net, test_input, ad.Zonotope)
    bounds.compute() 
    
    bounds.abstract_domain = ad.Hyperbox 
    bounds.bounds = [_.as_hyperbox() for _ in bounds.bounds]
    
    return bounds 

def get_zonotope_prespec(net, test_input):
    # Computes zonotopes properly
    bounds = PreactBounds(net, test_input, ad.Zonotope)
    bounds.compute() 
    return bounds 

def get_polybox_prespec(net, test_input):
    # Computes polytopes [Kolter-Wong thing]
    # (bounds.bounds is boxes, but we store the whole polytope too)
    bounds = PreactBounds(net, test_input, ad.Polytope)
    bounds.compute()
    return bounds



In [5]:
# E.g., what I'm saying about zonotopes vs polytopes:
zono_bounds = get_zonotope_prespec(bin_net, test_input)
poly_bounds = get_polybox_prespec(bin_net, test_input)

Using license file /home/matt/config/gurobi.lic
Academic license - for non-commercial use only


In [6]:
print("ZONO BOUNDS: [%.2f, %.2f]" % (zono_bounds.bounds[-1].lbs.item(), zono_bounds.bounds[-1].ubs.item()))
print("POLY BOUNDS: [%.2f, %.2f]" % (poly_bounds.bounds[-1].lbs.item(), poly_bounds.bounds[-1].ubs.item()))

ZONO BOUNDS: [-122.32, 176.48]
POLY BOUNDS: [-368.39, 555.43]


# Part 3: Actually doing the dual verification
Let's use the setup from the previous block where we want to lower bound the optimum of minimize `bin_net(x)` over all `x` in `test_input`


In [7]:
# For comparison, let's look at what happens when we use box-based inner minimizations
# (but intermediate bounds computed from zonotopes)

zonobox_bounds = get_zonobox_prespec(bin_net, test_input)
zonobox_dual = dn.NaiveDual(bin_net, test_input, preact_domain=ad.Hyperbox, 
                            prespec_bounds=zonobox_bounds, choice='naive')

optim_obj = optim.Adam(zonobox_dual.parameters(), lr=1e-2)


In [8]:
zonobox_out = zonobox_dual.dual_ascent(1000, verbose=25, optim_obj=optim_obj)

Iter 00 | Certificate: -122.32
Iter 25 | Certificate: -129.91
Iter 50 | Certificate: -126.21
Iter 75 | Certificate: -124.22
Iter 100 | Certificate: -122.38
Iter 125 | Certificate: -121.53
Iter 150 | Certificate: -119.32
Iter 175 | Certificate: -117.81
Iter 200 | Certificate: -116.55
Iter 225 | Certificate: -115.25
Iter 250 | Certificate: -113.37
Iter 275 | Certificate: -112.70
Iter 300 | Certificate: -110.20
Iter 325 | Certificate: -109.79
Iter 350 | Certificate: -106.75
Iter 375 | Certificate: -106.11
Iter 400 | Certificate: -103.39
Iter 425 | Certificate: -102.71
Iter 450 | Certificate: -101.23
Iter 475 | Certificate: -100.62
Iter 500 | Certificate: -100.98
Iter 525 | Certificate: -100.64
Iter 550 | Certificate: -100.45
Iter 575 | Certificate: -100.73
Iter 600 | Certificate: -101.35
Iter 625 | Certificate: -101.52
Iter 650 | Certificate: -101.34
Iter 675 | Certificate: -101.03
Iter 700 | Certificate: -101.19
Iter 725 | Certificate: -99.90
Iter 750 | Certificate: -100.92
Iter 775 | Ce

In [9]:
# On the other hand, we can do the same with zonotopes (no hyperbox cast)
# - run dual ascent for 1k iterations, and then start computing partition stuff 
zono_dual = dn.NaiveDual(bin_net, test_input, preact_domain=ad.Zonotope, 
                         choice='naive')

optim_obj = optim.Adam(zono_dual.parameters(), lr=1e-2)
zono_out = zono_dual.dual_ascent(1000, verbose=25, optim_obj=optim_obj)

Iter 00 | Certificate: -122.32
Iter 25 | Certificate: -126.29
Iter 50 | Certificate: -119.45
Iter 75 | Certificate: -116.10
Iter 100 | Certificate: -113.24
Iter 125 | Certificate: -110.10
Iter 150 | Certificate: -106.07
Iter 175 | Certificate: -102.26
Iter 200 | Certificate: -100.19
Iter 225 | Certificate: -99.63
Iter 250 | Certificate: -99.54
Iter 275 | Certificate: -99.05
Iter 300 | Certificate: -98.90
Iter 325 | Certificate: -99.50
Iter 350 | Certificate: -99.42
Iter 375 | Certificate: -98.71
Iter 400 | Certificate: -98.45
Iter 425 | Certificate: -98.49
Iter 450 | Certificate: -98.51
Iter 475 | Certificate: -98.11
Iter 500 | Certificate: -98.42
Iter 525 | Certificate: -98.36
Iter 550 | Certificate: -98.13
Iter 575 | Certificate: -97.87
Iter 600 | Certificate: -97.70
Iter 625 | Certificate: -98.12
Iter 650 | Certificate: -98.61
Iter 675 | Certificate: -97.37
Iter 700 | Certificate: -97.78
Iter 725 | Certificate: -97.77
Iter 750 | Certificate: -98.08
Iter 775 | Certificate: -97.85
Ite

In [10]:
# And we can examine the contribution of each subproblem to the total lagrangian 
zono_dual.lagrange_by_var(zono_dual.argmin())

OrderedDict([('x:0', tensor(2.0680, grad_fn=<DotBackward>)),
             ('z:0', tensor(-2.6312, grad_fn=<SubBackward0>)),
             ('x:1', tensor(-0.6389, grad_fn=<SubBackward0>)),
             ('z:1', tensor(-18.9046, grad_fn=<SubBackward0>)),
             ('x:2', tensor(-3.3282, grad_fn=<SubBackward0>)),
             ('z:2', tensor(-70.2211, grad_fn=<SubBackward0>)),
             ('x:3', tensor(-4.3652, grad_fn=<SubBackward0>)),
             ('output', tensor([-0.0924], grad_fn=<MulBackward0>)),
             ('total', tensor([-98.1137], grad_fn=<AddBackward0>))])

In [11]:
# Start partitioning by modifying the choice and partition kwargs attr 

print("Lagrange bounds using naive inner min: ", zono_dual.lagrangian(zono_dual.argmin()))


zono_dual.choice = 'partition' 
zono_dual.partition_kwargs = {'num_partitions': 8, 'partition_style': 'fixed'} 
# num partition is # of partitions per zonotope 
# partition_style 'fixed' saves partitions, whereas 'random' re-partitions every time
print("Lagrange bounds when you partition: ", zono_dual.lagrangian(zono_dual.argmin()))


Lagrange bounds using naive inner min:  tensor(-98.1137, grad_fn=<AddBackward0>)
Lagrange bounds when you partition:  tensor(-61.5584, grad_fn=<AddBackward0>)


In [12]:
# For this choice of dual variables lambda_, we can get bounds on the MIP subproblems (no partitioning) ...
# (note that further optimization of lambda_ will change these bounds)
est_bounds = zono_dual.lagrange_bounds({'TimeLimit': 10})

Changed value of parameter TimeLimit to 10.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 10.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 10.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [13]:
# X's are solved exactly, Z's are tuples with upper/lower bounds from MIP
est_bounds

{'x:0': tensor(2.0325, grad_fn=<DotBackward>),
 'z:0': (-1.4913844355166677, -0.556996407703031),
 'x:1': tensor(-0.6132, grad_fn=<DotBackward>),
 'z:1': (-14.927361286142926, -4.926206725345308),
 'x:2': tensor(-3.2226, grad_fn=<DotBackward>),
 'z:2': (-41.47969714289978, -23.379760264032754),
 'x:3': tensor(-4.5243, grad_fn=<DotBackward>),
 'output': tensor(-0.0924, grad_fn=<DotBackward>),
 'total_lb': tensor(-64.3185, grad_fn=<AddBackward0>),
 'total_ub': tensor(-35.2830, grad_fn=<AddBackward0>)}

# Part 4: Decomposition Objects
There's an improved lagrangian formulation using lagrangian splitting, but the same idea holds: you can switch box-based relu programming problems to zonotope-based ones. There's some theory that this provides Lagrangians that are no worse than previous bounds, but the main benefit is that iteration is quicker (but the formulation is slightly more tricky to reason about) 

This is contained in the `dual_decompose.DecompDual` class, and the API is basically the same.

In [14]:
zono_decomp = dd.DecompDual(bin_net, test_input,  preact_domain=ad.Zonotope, 
                            choice='naive', zero_dual=True)

# The only extra kwarg here is zero_dual, which initializes the dual variables 
# from the KW2017 paper, giving a slightly better initial bound 

zero_dual_bound = zono_decomp.lagrangian(zono_decomp.argmin())
zono_decomp = dd.DecompDual(bin_net, test_input,  preact_domain=ad.Zonotope, 
                            choice='naive', zero_dual=False)
init_dual_bound = zono_decomp.lagrangian(zono_decomp.argmin())

print("Zero dual bound: ", zero_dual_bound)
print("Init dual bound: ", init_dual_bound)


Zero dual bound:  tensor(-147.6271, grad_fn=<SubBackward0>)
Init dual bound:  tensor(-135.9158, grad_fn=<SubBackward0>)


In [15]:
optim_obj = optim.Adam(zono_decomp.parameters(), lr=1e-2)
zono_out = zono_decomp.dual_ascent(500, verbose=25, optim_obj=optim_obj)

optim_obj = optim.Adam(zono_decomp.parameters(), lr=1e-3)
zono_out = zono_decomp.dual_ascent(500, verbose=25, optim_obj=optim_obj)

Iter 00 | Certificate: -135.92
Iter 25 | Certificate: -102.54
Iter 50 | Certificate: -99.31
Iter 75 | Certificate: -98.35
Iter 100 | Certificate: -98.90
Iter 125 | Certificate: -99.00
Iter 150 | Certificate: -98.66
Iter 175 | Certificate: -98.83
Iter 200 | Certificate: -98.42
Iter 225 | Certificate: -98.86
Iter 250 | Certificate: -98.95
Iter 275 | Certificate: -99.16
Iter 300 | Certificate: -98.68
Iter 325 | Certificate: -98.78
Iter 350 | Certificate: -98.97
Iter 375 | Certificate: -98.92
Iter 400 | Certificate: -98.40
Iter 425 | Certificate: -98.59
Iter 450 | Certificate: -98.60
Iter 475 | Certificate: -98.51
Iter 00 | Certificate: -98.68
Iter 25 | Certificate: -96.30
Iter 50 | Certificate: -96.14
Iter 75 | Certificate: -96.15
Iter 100 | Certificate: -96.10
Iter 125 | Certificate: -96.11
Iter 150 | Certificate: -96.12
Iter 175 | Certificate: -96.09
Iter 200 | Certificate: -96.12
Iter 225 | Certificate: -96.09
Iter 250 | Certificate: -96.14
Iter 275 | Certificate: -96.10
Iter 300 | Cer

In [16]:
zono_decomp.choice = 'partition' 
zono_decomp.partition_kwargs = {'partition_style': 'fixed', 
                                'num_partitions': 16}
optim_obj = optim.Adam(zono_decomp.parameters(), lr=1e-2)

zono_out = zono_decomp.dual_ascent(25, verbose=1, optim_obj=optim_obj)

Iter 00 | Certificate: -73.29
Iter 01 | Certificate: -78.98
Iter 02 | Certificate: -77.50
Iter 03 | Certificate: -74.22
Iter 04 | Certificate: -72.28
Iter 05 | Certificate: -73.28
Iter 06 | Certificate: -74.11
Iter 07 | Certificate: -73.23
Iter 08 | Certificate: -72.03
Iter 09 | Certificate: -72.15
Iter 10 | Certificate: -72.34
Iter 11 | Certificate: -71.93
Iter 12 | Certificate: -71.43
Iter 13 | Certificate: -71.39
Iter 14 | Certificate: -71.61
Iter 15 | Certificate: -71.44
Iter 16 | Certificate: -71.08
Iter 17 | Certificate: -70.75
Iter 18 | Certificate: -70.40
Iter 19 | Certificate: -70.33
Iter 20 | Certificate: -70.25
Iter 21 | Certificate: -70.09
Iter 22 | Certificate: -69.96
Iter 23 | Certificate: -69.89
Iter 24 | Certificate: -69.78


In [17]:
zono_decomp.lagrange_bounds({'TimeLimit': 10})

Changed value of parameter TimeLimit to 10.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 10.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 10.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


{'P1': (-0.05238720476019054, 1.1855643692595321),
 'P3': (-16.329288752416232, -3.9014411342751885),
 'P5': (-47.098353389437726, -24.254115707336513),
 'total_lb': -63.480029346614145,
 'total_ub': -26.96999247235217}

# 5. Interacting with Zonotopes/Partitioning
Finally, we can consider the various ways we can partition/merge zonotope partitions.
First I'll go over how to modify the partitioning of the dual objects, then how to do this for zonotopes in general

In [18]:
# Consider an existing dual object with some partitions 
print(zono_dual.choice)
print(zono_dual.partition_kwargs.keys())

partition
dict_keys(['num_partitions', 'partition_style', 'partitions'])


In [19]:
# Examining the actual partitions: 
# It's a dict with keys pointing to each layer's zonotope
zono_dual.partition_kwargs['partitions'].keys()

dict_keys([1, 3, 5])

In [20]:
# And each layer is a list of 
print(type(zono_dual.partition_kwargs['partitions'][1]))

# Where each element is a tuple like (idxs_of_original, zonotope)
zono_dual.partition_kwargs['partitions'][1][0]

<class 'list'>


([1,
  19,
  39,
  51,
  56,
  62,
  85,
  87,
  88,
  104,
  112,
  114,
  118,
  137,
  138,
  156,
  159,
  165,
  170,
  174,
  183,
  184,
  187,
  188,
  190,
  196,
  201,
  206,
  211,
  231,
  248,
  254],
 <abstract_domains.Zonotope at 0x7f96e8958780>)

In [21]:
# The only things you'd probably want to do with a dual object is to 
# 1. reset the partitions 
# 2. merge existing partitions together 
zono_dual.partition_kwargs['partitions'] = None  # resents the partitions 


zono_dual.argmin() # Will remake the partitions 
zono_dual.shrink_partitions(4) # now 4 partitions per zonotope

In [22]:
##### And to examine the individual zonotopes #####
zono_ex = zono_dual.preact_bounds.bounds[1]
zono_ex

In [23]:
# You can get the center, generator, element-wise lower and upper bounds like 
print(zono_ex.center.shape, zono_ex.generator.shape)
print(zono_ex.lbs, zono_ex.ubs)

torch.Size([256]) torch.Size([256, 784])
tensor([ -4.0201,  -1.4783,  -1.9837,  -3.4214,  -5.2391,  -0.4408,  -5.8371,
         -5.9935,  -4.1471,  -5.4295,  -2.6514,  -5.7663,  -7.1188,  -4.9403,
          1.5643,  -2.4075,   0.5999,  -3.2104,  -3.7678,  -5.1422,  -4.1153,
         -5.5919,  -6.8462,  -0.2810,  -0.7184,  -5.7582,  -4.1536,  -2.3583,
         -4.0887,  -3.4226,  -6.9798,  -0.8708,  -5.0678,  -1.0506,  -5.9683,
         -5.7655,  -3.1604,  -5.1112,  -4.7740,  -2.6093,  -8.5837,  -7.0995,
         -5.7920,  -5.2219,  -4.7178,  -1.7338,  -6.2872,  -4.8799,  -0.2524,
         -1.4707,  -3.4506,  -3.1170,  -4.6625,  -7.1756,  -5.1780,  -4.4660,
         -7.4255,  -3.1039,  -4.7040,  -2.4035,  -7.5073,  -6.5595,  -4.8489,
         -5.0170,  -7.3768,  -5.7802,  -5.7434,  -9.0429,  -4.9368,   0.3265,
         -2.2572,  -2.5454,  -0.6178,  -1.2705,  -5.3893,  -4.0515,   0.4418,
         -5.6087,  -5.2046,  -6.3975,  -3.6302,   1.5571,  -7.9715,  -3.5816,
         -3.8302,  -4.5

In [24]:
# To solve a vanilla linear program over the zonotope:
zono_ex.solve_lp(torch.ones_like(zono_ex.lbs), get_argmin=True)

(tensor(-485.0855, grad_fn=<DotBackward>),
 tensor([-1.7733,  0.8088,  0.1956, -1.8161, -3.4528,  1.5994, -3.8733, -3.6302,
         -2.4468, -3.7438,  0.3520, -3.5095, -4.4656, -2.4714,  4.1153, -1.3404,
          3.1901, -0.5386, -1.0052, -3.8602, -2.2105, -3.4108, -4.5283,  2.3665,
          1.8820, -2.9261, -2.1691,  0.5874, -1.6264, -0.4566, -4.4463,  1.4133,
         -2.9252,  1.3198, -4.0773, -4.3123, -0.6865, -2.8027, -2.8688, -0.3986,
         -5.5912, -4.9252, -2.3303, -3.0548, -2.6407,  0.5823, -3.7131, -3.2493,
          2.6870,  0.7620, -0.7740, -0.9270, -2.2163, -5.1500, -2.7819, -1.8592,
         -4.8217, -0.3862, -3.1467,  0.4431, -5.7661, -3.9550, -3.2796, -3.1084,
         -5.5714, -3.8127, -3.6871, -6.4311, -2.3189,  2.6746, -1.0913,  0.2318,
          1.7433,  1.2243, -3.2686, -2.1211,  3.2559, -3.0997, -3.6713, -4.6153,
         -1.2332,  3.8910, -5.3573, -0.8602, -1.3464, -2.3133, -3.0320, -3.9914,
         -0.7533, -1.4813,  0.7830, -1.2929, -2.5817,  1.4208, -0.

In [25]:
# To solve a relu program: min_z c1@z + c2@relu(z)... 
c1 = torch.ones_like(zono_ex.lbs)
c2 = -torch.ones_like(zono_ex.lbs)
zono_ex.solve_relu_mip(c1, c2, apx_params={'TimeLimit': 10}, verbose=True)

(-585.984829138688,
 <gurobi.Model MIP instance Unnamed: 871 constrs, 1451 vars, Parameter changes: TimeLimit=10.0, OutputFlag=0>)

In [26]:
# To create partitions:
parts = zono_ex.make_random_partitions(10)
parts

[([6,
   13,
   14,
   21,
   31,
   33,
   41,
   45,
   50,
   56,
   59,
   75,
   108,
   110,
   121,
   123,
   137,
   148,
   153,
   161,
   188,
   203,
   205,
   214,
   233,
   253],
  <abstract_domains.Zonotope at 0x7f96787f7da0>),
 ([8,
   18,
   28,
   39,
   47,
   51,
   66,
   98,
   109,
   119,
   128,
   133,
   141,
   143,
   162,
   166,
   170,
   177,
   179,
   189,
   220,
   221,
   245,
   249,
   251,
   252],
  <abstract_domains.Zonotope at 0x7f96787f7358>),
 ([10,
   17,
   32,
   38,
   52,
   70,
   73,
   80,
   91,
   111,
   112,
   113,
   129,
   139,
   150,
   158,
   160,
   167,
   175,
   194,
   196,
   201,
   204,
   206,
   222,
   225],
  <abstract_domains.Zonotope at 0x7f96787f7780>),
 ([3,
   5,
   9,
   23,
   46,
   71,
   78,
   86,
   87,
   97,
   114,
   120,
   157,
   163,
   168,
   171,
   185,
   191,
   200,
   209,
   216,
   237,
   243,
   250,
   254,
   255],
  <abstract_domains.Zonotope at 0x7f96787f7390>),
 ([1,
  

In [27]:
# and to merge partitions back together 
half_parts_a = ad.Zonotope.merge_partitions(parts[::2])
half_parts_b = ad.Zonotope.merge_partitions(parts[1::2])
half_parts_a

[([6,
   13,
   14,
   21,
   31,
   33,
   41,
   45,
   50,
   56,
   59,
   75,
   108,
   110,
   121,
   123,
   137,
   148,
   153,
   161,
   188,
   203,
   205,
   214,
   233,
   253,
   10,
   17,
   32,
   38,
   52,
   70,
   73,
   80,
   91,
   111,
   112,
   113,
   129,
   139,
   150,
   158,
   160,
   167,
   175,
   194,
   196,
   201,
   204,
   206,
   222,
   225,
   1,
   2,
   4,
   15,
   20,
   40,
   42,
   55,
   60,
   61,
   76,
   81,
   82,
   90,
   99,
   115,
   118,
   164,
   182,
   192,
   228,
   240,
   241,
   246,
   247,
   248,
   0,
   19,
   27,
   34,
   44,
   53,
   58,
   67,
   85,
   89,
   92,
   96,
   101,
   124,
   126,
   131,
   136,
   145,
   155,
   159,
   183,
   217,
   223,
   227,
   231,
   11,
   16,
   22,
   24,
   35,
   48,
   57,
   74,
   88,
   93,
   117,
   134,
   151,
   172,
   174,
   195,
   198,
   202,
   208,
   211,
   212,
   215,
   229,
   236,
   239],
  <abstract_domains.Zonotope at 0x7f96

In [28]:
#... and that's all, I think